In [1]:
%matplotlib inline
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import IPython.display
import collections
from collections import Counter
import seaborn as sns

from pathlib import Path
import pretty_midi
import librosa.display
import glob
import datetime

from scipy.spatial import distance

from music21 import *

import tensorflow as tf

%load_ext autoreload
%autoreload 2

folder = 'train/'


In [2]:
## krumhansl schmuckler

# adapted from https://gist.github.com/bmcfee/1f66825cef2eb34c839b42dddbad49fd
import scipy.linalg
import scipy.stats

def ks_key(X, weights='kessler'):
    '''Estimate the key from a pitch class distribution
    
    Parameters
    ----------
    X : np.ndarray, shape=(12,)
        Pitch-class energy distribution.  Need not be normalized
        
    Returns
    -------
    major : np.ndarray, shape=(12,)
    minor : np.ndarray, shape=(12,)
    
        For each key (C:maj, ..., B:maj) and (C:min, ..., B:min),
        the correlation score for `X` against that key.
    '''
    X = scipy.stats.zscore(X)
    
    # Coefficients from Kumhansl and Schmuckler
    # as reported here: http://rnhart.net/articles/key-finding/
    
    # weights as https://musicinformationretrieval.wordpress.com/2017/02/24/state-of-the-art-2/
    if weights == 'kessler':
        major = np.asarray([6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.88])
        minor = np.asarray([6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17])
    elif weights == 'schmuckler':
        major = np.asarray([6.35, 2.33, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.88])
        minor = np.asarray([6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17])
    elif weights == 'aarden':
        major = np.asarray([17.7661, 0.145624, 14.9265, 0.160186, 19.8049, 11.3587, 0.281248, 22.062, 0.145624, 8.15494, 0.232998, 4.95122])
        minor = np.asarray([18.2648, 0.737619, 14.0499, 16.8599, 0.702494, 14.4362, 0.702494, 18.6161, 4.56621, 1.93186, 7.37619, 1.75623])
    elif weights == 'sapp':
        major = np.asarray([2, 0, 1, 0, 1, 1, 0, 2, 0, 1, 0, 1])
        minor = np.asarray([2, 0, 1, 1, 0, 1, 0, 2, 1, 0, 0.5, 0.5])
    elif weights == 'bellman':
        major = np.asarray([16.8, 0.86, 12.95, 1.41, 13.49, 11.93, 1.25, 20.28, 1.8, 8.04, 0.62, 10.57])
        minor = np.asarray([18.16, 0.69, 12.99, 13.34, 1.07, 11.15, 1.38, 21.07, 7.49, 1.53, 0.92, 10.21])
    elif weights == 'temperley':
        major = np.asarray([0.748, 0.06, 0.488, 0.082, 0.67, 0.46, 0.096, 0.715, 0.104, 0.366, 0.057, 0.4])
        minor = np.asarray([0.712, 0.84, 0.474, 0.618, 0.049, 0.46, 0.105, 0.747, 0.404, 0.067, 0.133, 0.33])
        
        
    major = scipy.stats.zscore(major)
    minor = scipy.stats.zscore(minor)
    
    # Generate all rotations of major
    major = scipy.linalg.circulant(major)
    minor = scipy.linalg.circulant(minor)
    
    major = major.T.dot(X)
    minor = minor.T.dot(X)
    
    major_max = np.argmax(major)
    minor_max = np.argmax(minor)
    return major_max if np.max(major) >= np.max(minor) else minor_max+12

In [3]:
## Spiral Array

def chroma_to_fifth(num):
    if num > 11:
        return ((num - 12) * 7) % 12 + 12
    else:
        return (num * 7) % 12
def fifth_to_chroma(num):
    return chroma_to_fifth(num)

keys = ['C','G','D','A','E','B','F#','C#','G#','D#','A#', 'F',
        'C-','G-','D-','A-','E-','B-','F#-','C#-','G#-','D#-','A#-', 'F-']
keys_dict = dict(enumerate(keys))
# keys_dict
keys_dict_reverse = {v: k for k, v in keys_dict.items()}
# keys_dict_reverse
keys_chroma = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B',
        'C-','C#-','D-','D#-','E-','F-','F#-','G-','G#-','A-','A#-','B-']
keys_chroma_dict = dict(enumerate(keys_chroma))
keys_chroma_dict_reverse = {v: k for k, v in keys_chroma_dict.items()}
# keys_chroma_dict_reverse

def P(k, r=1, h=np.sqrt(2/15)):
    return np.array([r*np.sin(k*np.pi/2), r*np.cos(k*np.pi/2), k*h])

def C_M(k, w=np.array([0.516, 0.315, 0.168])):
    return w[0]*P(k) + w[1]*P(k+1) + w[2]*P(k+4)

def C_m(k, w=np.array([0.516, 0.315, 0.168])):
    return w[0]*P(k) + w[1]*P(k+1) + w[2]*P(k-3)

def T_M(k, w=np.array([0.516, 0.315, 0.168])):
    return w[0]*C_M(k) + w[1]*C_M(k+1) + w[2]*C_M(k-1)

def T_m(k, w=np.array([0.516, 0.315, 0.168]), alpha=1, beta=0):
    t1 = w[0]*C_m(k)
    t2 = w[1]*(alpha*C_M(k+1) + (1-alpha)*C_m(k+1))
    t3 = w[2]*(beta*C_m(k-1) + (1-beta)*C_M(k-1))
    return t1 + t2 + t3

def getPitch(pitch):
    return (pitch+6)%12 if pitch%2 == 1 else pitch%12
    
def centerOfEffect(notes):
    notes_and_durations = [(getPitch(note.pitch), note.end - note.start) for note in notes]
    duration_sum = sum(duration for pitch, duration in notes_and_durations)
    c = sum(P(pitch) * duration for pitch, duration in notes_and_durations) / duration_sum
    return c


T = np.array([T_M(k) for k in range(12)] + [T_m(k) for k in range(12)])
C = np.array([C_M(k) for k in range(12)] + [C_m(k) for k in range(12)])
Parr = np.array([P(k) for k in range(12)])

def key_sa(pmobject):
    # combine all notes
    allnotes = [note for track in pmobject.instruments if track.is_drum==False for note in track.notes]
    c = centerOfEffect(allnotes)
    distances = distance.cdist([c], T)
    top_indices = np.argsort(distances)[0]
    closest_index = top_indices[0]
    top_indices = [keys_dict[ind] for ind in top_indices] 
    return chroma_to_fifth(closest_index) #, top_indices

def key_iterative(pmobject):
    predictions = []
    notes = [note for track in pmobject.instruments if track.is_drum==False for note in track.notes]
    for i in range(1,len(notes)):
        c = centerOfEffect(notes[:i])
        distances = distance.cdist([c], T)
        top_indices = np.argsort(distances)[0]
        closest_index = top_indices[0]
        predictions.append(chroma_to_fifth(closest_index))
    return predictions

def key_sa2(pmobject):
    predictions = key_iterative(pmobject)
    count = Counter(predictions)
    return max(count.items(), key=(lambda key: stats[key]))[0]



## Testing

In [4]:
def key_to_name(key_number):
    return pretty_midi.key_number_to_key_name(key_number)

def keynr(key):
    return pretty_midi.key_name_to_key_number(key)
       
def isFifth(realkey,predkey):
    return (realkey - predkey)%12 == 7 or (predkey - realkey)%12 == 7
        
def isRelative(realkey,predkey):
    if realkey > 11 and (realkey - 9) % 12 == predkey:
        return True
    elif predkey > 11 and (predkey - 9) % 12 == realkey:
        return True
    else:
        return False
        
def isParallel(realkey,predkey):
    return realkey%12 == predkey%12

In [6]:
#folder = "datasets/lmd_full/2/"
folder = "datasets/bachwtcall/" #source: https://ccrma.stanford.edu/~craig/keyscape/bach-wtc-new/ accessed: 18.07.19
t = time.time()

# find keys
files = [file for file in os.listdir(folder) if '.mid' in file.lower()]

files = files[:1000]
withoutC = False

verbose = False
keys=[]
for i, file in enumerate(files):
    try:
        pm = pretty_midi.PrettyMIDI(folder + file)
    except:
        continue

    pm = pretty_midi.PrettyMIDI(folder + file)
    if pm.key_signature_changes != []:
        try:
            kskey = ks_key(pm.get_pitch_class_histogram(), 'aarden')
            sakey = key_sa(pm)
            pmkeys = [x.key_number for x in pm.key_signature_changes]
            pmkey = pmkeys[0]
#             print(pmkey,kskey,sakey)
            if withoutC and pmkey == 0:
                continue
            keys.append([pmkey,kskey,sakey])
#             print(key_to_name(pmkey), '\t', key_to_name(int(algkey)), '\t', file, '\t', pmkeys)
        except:
            pass
    

total = 0
correct,fifth,relative,parallel = 0,0,0,0
sacorrect,safifth,sarelative,saparallel = 0,0,0,0
for key in keys:
    realkey = key[0]
    kskey = key[1]
    sakey = key[2]
    total += 1
    # ks
    if realkey == kskey:
        correct += 1
    elif isFifth(realkey,kskey):
        fifth += 1
    elif isRelative(realkey,kskey):
        relative += 1
    elif isParallel(realkey,kskey):
        parallel += 1
    # sa
    if realkey == sakey:
        sacorrect += 1
    elif isFifth(realkey,sakey):
        safifth += 1
    elif isRelative(realkey,sakey):
        sarelative += 1
    elif isParallel(realkey,sakey):
        saparallel += 1
        

print("time", np.round(time.time()-t,2), "s")
print("Nr of files:", total)
if verbose:
    print('ks')
    print(correct, relative, fifth, parallel, total)
    print((correct+relative)/total, correct/total, relative/total, fifth/total, parallel/total, (total-correct-fifth-relative-parallel)/total)
    print('sa')
    print(sacorrect, sarelative, safifth, saparallel, total)
    print((sacorrect+sarelative)/total, sacorrect/total, sarelative/total, safifth/total, saparallel/total, (total-sacorrect-safifth-sarelative-saparallel)/total)

print()
print("KrumhanslSchmuckler with Aarden weights has", np.round(correct/total,2), "correct")
print("and predicted the relative minor for", np.round(relative/total,2), "which amounts to a score of ", np.round((correct+relative)/total,2))
print()
print("SpiralArray has", np.round(sacorrect/total,2), "correct")
print("and predicted the relative minor for", np.round(sarelative/total,2), "which amounts to a score of ", np.round((sacorrect+sarelative)/total,2))



time 9.01 s
Nr of files: 90

KrumhanslSchmuckler with Aarden weights has 0.83 correct
and predicted the relative minor for 0.14 which amounts to a score of  0.98

SpiralArray has 0.28 correct
and predicted the relative minor for 0.13 which amounts to a score of  0.41


## Results

Results on 18.07.19 on Bach Fugues from https://ccrma.stanford.edu/~craig/keyscape/bach-wtc-new/

time 11.75 s

Nr of files: 90

KrumhanslSchmuckler with Aarden weights has 0.83 correct
and predicted the relative minor for 0.14 which amounts to a score of  0.98

SpiralArray has 0.28 correct
and predicted the relative minor for 0.13 which amounts to a score of  0.41

Results on lmd_full/2 

time 602.91 s

Nr of files: 169

KrumhanslSchmuckler with Aarden weights has 0.62 correct
and predicted the relative minor for 0.18 which amounts to a score of  0.8

SpiralArray has 0.37 correct
and predicted the relative minor for 0.04 which amounts to a score of  0.41